# PIC-SURE API use-case: quick analysis on COPDgene data

This is a tutorial notebook, aimed to be quickly up and running with the python PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE python API 
### What is PIC-SURE? 

<!--img src="./img/PIC-SURE_logo.png" width= "360px"> -->

As part of the BioData Catalyst initiative, the Patient Information Commons Standard Unification of Research Elements (PIC-SURE) platform has been integrating clinical and genomic datasets from multiple TOPMed and TOPMed related studies funded by the National Heart Lung and Blood Institute (NHLBI). 

Original data exposed through the PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and exposes the different study datasets in a single tabular format. By easing the process of data extraction, it allows investigators to focus on downstream analysis and to facilitate reproducible science.

Currently, only phenotypic variables are accessible through the PIC-SURE API, but access to genomic variables is coming soon.

### More about PIC-SURE
The API is available in two different programming languages, python and R, enabling investigators to query the databases the same way using any of those languages.

PIC-SURE is a larger project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface that allows researchers to explore variables across multiple studies, filter patients that match criteria, and create cohorts from this interactive exploration.

The python API is actively developed by the Avillach-Lab at Harvard Medical School.

PIC-SURE API GitHub repo:
* https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-client



 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the NHLBI_BioData_Catalyst ```README.md``` file. It contains an explanation about how to get a security token, which is mandatory to access the databases.**

# Environment set-up

### Pre-requisite
- python 3.6 or later
- pip python package manager, already available in most systems with a python interpreter installed ([pip installation instructions](https://pip.pypa.io/en/stable/installing/))

### Packages installation

Installation of the packages listed in the `requirements.txt` file, as well as the two components of the PIC-SURE API from GitHub: the PIC-SURE adapter and the PIC-SURE Client.

In [ ]:
!cat requirements.txt

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git

Import all the external dependencies, as well as user-defined functions stored in the `python_lib` folder

In [ ]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureClient
import PicSureBdcAdapter

from python_lib.utils import get_multiIndex_variablesDict, joining_variablesDict_onCol

##### Setting the display parameter for tables and plots

In [ ]:
# Pandas DataFrame display options
pd.set_option("max.rows", 100)

# Matplotlib display parameters
plt.rcParams["figure.figsize"] = (14,8)
font = {'weight' : 'bold',
        'size'   : 12}
plt.rc('font', **font)

## Connecting to a PIC-SURE resource

The following is required to get access to data through the PIC-SURE API: 
- a network URL
- a resource id, and 
- a user-specific security token.

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"

In [ ]:
with open(token_file, "r") as f:
    my_token = f.read()

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)
adapter = PicSureBdcAdapter.Adapter(connection)
resource = adapter.useResource(resource_id)

Two objects are created here: a `connection` and a `resource` object.

As we will only be using one single resource, **the `resource` object is actually the only one we will need to proceed with data analysis hereafter**. 

It is connected to the specific data source ID we specified, and enables to query and retrieve data from this database.

## Getting help with the PIC-SURE API

Each object exposed by the PicSureBdcHpds library has a `help()` method. Calling it without parameters will print out  information about functionalities of this object.

In [ ]:
resource.help()

For instance, this output tells us that this `resource` object has 3 methods, and it gives a quick definition of those methods. 

## Using the *variables dictionary*

Once a connection to the desired resource has been established, we first need to understand which variables are available in the database. To this end, we will use the `dictionary` method of the `resource` object.

A `dictionary` instance enables us to retrieve matching records by searching for a specific term, or to retrieve information about all the available variables, using the `find()` method. For instance, looking for variables containing the term `COPD` in their names is done this way: 

In [ ]:
dictionary = resource.dictionary()
dictionary_search = dictionary.find("COPD")

Subsequently, objects created by the `dictionary.find` method expose the search results via 4 different methods: `.count()`, `.keys()`, `.entries()`, and `.DataFrame()`. 

In [ ]:
pprint({"Count": dictionary_search.count(), 
        "Keys": dictionary_search.keys()[0:5],
        "Entries": dictionary_search.entries()[0:5]})

In [ ]:
dictionary_search.DataFrame().head()

**The `.DataFrame()` method enables us to get the result of the dictionary search in a pandas DataFrame format. This way, it allows us to:** 


* Use the various information exposed in the dictionary (patient count, variable type ...) as criteria for variable selection.
* Use the row names of the DataFrame to get the actual variable names to be used in the query, as shown below.

Variable names aren't very pratical to use right away for two reasons:
1. Very long
2. Presence of backslashes that prevent from copy-pasting. 

However, retrieving the dictionary search result in the form of a dataframe can help to deal with this, as below:

In [ ]:
plain_variablesDict = resource.dictionary().find().DataFrame()
plain_variablesDict.shape

Using the `dictionary.find()` function without arguments will return every entry, as shown in the help documentation.

In [ ]:
resource.dictionary().help()

In [ ]:
plain_variablesDict.iloc[10:20,:]

The dictionary currently returned by the API provides information about the variables, such as:
- observationCount: number of entries with non-null value
- categorical: type of the variables, True if strings, False if numerical
- min/max: only provided for numerical variables
- HpdsDataType: 'phenotypes' or 'genotypes'. Currently, the API only expsoses'phenotypes' variables

### Export Full Data Dictionary to CSV

In order to export the data dictionary first we will create a Pandas dataframe called fullVariableDict

In [ ]:
fullVariableDict = resource.dictionary().find().DataFrame()

Check that the fullVariableDict dataframe contains some values.

In [ ]:
fullVariableDict.iloc[0:3,:]

In [ ]:
fullVariableDict.to_csv('data_dictionary.csv')

You should now see a data_dictionary.csv in the file explorer.

#### Variable dictionary + pandas multiIndex

We can use a simple user-defined function (`get_multiIndex_variablesDict`) to add a little more information to the variable dictionary and to simplify working with variables names. It takes advantage of pandas MultiIndex functionality [see pandas official documentation on this topic](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Although not an official feature of the API, such functionality illustrates how to quickly select groups of related variables.

Printing the 'multiIndexed' variable Dictionary allows to quickly see the tree-like organisation of the variable names. Moreover, original and simplified variable names are now stored respectively in the "varName" and "simplified_varName" columns (simplified variable names is simply the last component of the variable name, that is usually the most informative to know what each variable is about).

In [ ]:
variablesDict = get_multiIndex_variablesDict(plain_variablesDict)

In [ ]:
variablesDict

In [ ]:
# Now that we have seen how our entire dictionary looked, we limit the number of lines to be displayed for the future outputs
pd.set_option("max.rows", 50)

Below is a simple example to illustrate the simplicity of use a multiIndex dictionary. Let's say we are interested in every variables pertaining to the "Medical history" and "Medication history" subcategories.

In [ ]:
mask_medication = variablesDict.index.get_level_values(2) == "Medication History"
mask_medical = variablesDict.index.get_level_values(2) == "Medical History"
medication_history_variables = variablesDict.loc[mask_medical | mask_medication,:]
medication_history_variables

Although pretty simple, it can be easily combined with other filters to quickly select desired group of variables.

## Querying and retrieving data

The second cornerstone of the API is the `query` object. It is how we retrieve data from the resource.

First, we need to create a query object.

In [ ]:
my_query = resource.query()

The query object has several methods that enable to build a query.

- The `query.select.add()` method accepts variable names as string or list of strings as argument, and will allow the query to return all variables included in the list, without any record (ie subjects/rows) subsetting.

- The `query.require.add()` method accepts variable names as string or list of strings as argument, and will allow the query to return all the variables passed, and only records that do not contain any null values for those variables.

- The `query.anyof.add()` method accepts variable names as string or list of strings as argument, and will allow the query to return all variables included in the list, and only records that do contain at least one non-null value for those variables.

- The `query.filter.add()` method accepts a variable name as argument, plus additional values to filter on that given variable. The query will return this variable and only the records that do match this filter criteria.

All 4 methods can be combined when building a query. The record eventually returned by the query have to meet all the different specified filters.

#### Building the query

In [ ]:
mask = variablesDict["simplified_name"] == "How old were you when you completely stopped smoking? [Years old]"
yo_stop_smoking_varname = variablesDict.loc[mask, "name"] 

In [ ]:
mask_cat = variablesDict["categorical"] == True
mask_count = variablesDict["observationCount"].between(100,2000)
varnames = variablesDict.loc[mask_cat & mask_count, "name"]

In [ ]:
my_query.filter().add(yo_stop_smoking_varname, min=20, max=70)
my_query.select().add(varnames[:50])

 ## Selecting Consent Groups

Sometimes it can be necessary to limit results to a group of patients that have provided common consent types. By default, PIC-SURE will enforce limits to the consents that each researcher has individually been authorized for, however it may be desirable to further restrict the results. To view the available consent groups, you can use the resource.list_consents() function on the resource connection object.

In [ ]:
resource.list_consents()

In order to update the values, the existing list needs to be cleared first, then replaced. (phs000179.c2 is one consent code used in the COPDGene study.)

In [ ]:
my_query.filter().delete("\\_consents\\")

In [ ]:
my_query.filter().add("\\_consents\\", ['phs000179.c2'])

### Retrieving the data

Once our query object is finally built, we use the `getResultsDataFrame` function to retrieve the data corresponding to our query

In [ ]:
query_result = my_query.getResultsDataFrame(low_memory=False)

In [ ]:
query_result.shape

In [ ]:
query_result.tail()

In [ ]:
query_result[yo_stop_smoking_varname].plot.hist(legend=None, title= "Age stopping smoking", bins=15)

## Retrieving data from query run through PIC-SURE UI

It is possible for you to retrieve the results of a query that you have previously run using the PIC-SURE UI. To do this you must "select data for export", then select the information that you want the query to return and then click "prepare data export". Once the query is finished executing, a group of buttons will be presented.  Click the "copy query ID to clipboard" button to copy your unique query identifier so you can paste it into your notebook.


Paste your query's ID into your notebook and assign it to a variable.  You then use the `query.getResults(resource, yourQueryUUID)` function with an initialized resource object to retrieve the data from your query as shown below.


The screenshot below shows the button of interest in the PIC-SURE UI. It shows that the previously run query has a DataSetID of `dce08fab-98d3-434a-937a-cb583679efe8`. At this point a copy-paste process is used to provide the DataSetID to the API, as shown in the example code below.  To run this code you must replace the example query ID with a query ID from a query that you have run in the PIC-SURE API.

<img src="https://drive.google.com/uc?id=1kxFLxjEdMfkF4HjdWBaNju0PyMrYxGR0">


In [ ]:
# To run this using your notebook you must replace it with the ID value of a query that you have run.
DataSetID = '<<replace with your QuerySetID>>'

In [ ]:
%%capture
results = resource.retrieveQueryResults(DataSetID)

In [ ]:
from io import StringIO
df_UI = pd.read_csv(StringIO(results), low_memory=False)

In [ ]:
df_UI.head()